In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer
!pip install numpy==1.26.4

/Users/diallothiernosouleymane/Desktop/AppleTV/backend/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [33]:
import numpy as np
from sklearn.cluster import KMeans

In [3]:
!pip install sentence-transformers


[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [4]:
df=pd.read_csv("../archive/titles.csv")

In [5]:
df.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,tm1300,A Charlie Brown Christmas,MOVIE,When Charlie Brown complains about the overwhe...,1965,G,25,"['comedy', 'drama', 'family', 'music', 'animat...",['US'],NaN,tt0059026,8.3,40328.0,10.848,7.688
1,tm71152,"It's the Great Pumpkin, Charlie Brown",MOVIE,"This classic ""Peanuts"" tale focuses on the thu...",1966,G,25,"['animation', 'comedy', 'family']",['US'],NaN,tt0060550,8.2,20299.0,12.458,7.474
2,tm2562,A Charlie Brown Thanksgiving,MOVIE,"Turkey, cranberries, pumpkin pie... and the Pe...",1973,G,25,"['family', 'animation', 'comedy']",['US'],NaN,tt0068359,7.7,11406.0,12.591,7.311
3,tm125274,"Be My Valentine, Charlie Brown",MOVIE,It's Valentine's Day again and Charlie Brown d...,1975,G,25,"['animation', 'family', 'romance', 'comedy']",['US'],NaN,tt0072687,7.4,2777.0,6.981,7.000
4,tm106607,"It's the Easter Beagle, Charlie Brown",MOVIE,"Charlie Brown, Linus and the entire Peanuts ga...",1974,G,25,"['family', 'animation', 'comedy']",['US'],NaN,tt0071679,7.5,3001.0,6.172,6.600


In [6]:
df.columns

Index(['id', 'title', 'type', 'description', 'release_year',
       'age_certification', 'runtime', 'genres', 'production_countries',
       'seasons', 'imdb_id', 'imdb_score', 'imdb_votes', 'tmdb_popularity',
       'tmdb_score'],
      dtype='object')

In [7]:
df["seasons"]=df["seasons"].fillna("0")

In [8]:
df.isna().sum()

id                       0
title                    0
type                     0
description              0
release_year             0
age_certification       11
runtime                  0
genres                   0
production_countries     0
seasons                  0
imdb_id                  1
imdb_score               3
imdb_votes               4
tmdb_popularity          0
tmdb_score               3
dtype: int64

In [9]:
#cherchons le mode
mode_value = df['age_certification'].mode()[0]
print(mode_value)





TV-MA


In [10]:
df["age_certification"]=df["age_certification"].fillna(mode_value)

In [11]:
mode_imdbScore = df['imdb_score'].mode()[0]
print(mode_imdbScore)

7.7


In [12]:
df["imdb_score"]=df["imdb_score"].fillna(mode_imdbScore)

In [13]:
mode_tmdbScore = df['tmdb_score'].mode()[0]
print(mode_tmdbScore)

7.0


In [14]:
df["tmdb_score"]=df["tmdb_score"].fillna(mode_tmdbScore)

In [15]:
df.columns

Index(['id', 'title', 'type', 'description', 'release_year',
       'age_certification', 'runtime', 'genres', 'production_countries',
       'seasons', 'imdb_id', 'imdb_score', 'imdb_votes', 'tmdb_popularity',
       'tmdb_score'],
      dtype='object')

In [16]:
data=df[["release_year",'runtime', 'genres','seasons','production_countries','age_certification','imdb_score','tmdb_score']]

In [17]:
data.head()

,release_year,runtime,genres,seasons,production_countries,age_certification,imdb_score,tmdb_score
0,1965,25,"['comedy', 'drama', 'family', 'music', 'animat...",0,['US'],G,8.3,7.688
1,1966,25,"['animation', 'comedy', 'family']",0,['US'],G,8.2,7.474
2,1973,25,"['family', 'animation', 'comedy']",0,['US'],G,7.7,7.311
3,1975,25,"['animation', 'family', 'romance', 'comedy']",0,['US'],G,7.4,7.000
4,1974,25,"['family', 'animation', 'comedy']",0,['US'],G,7.5,6.600


In [18]:
data.isna().sum()

release_year            0
runtime                 0
genres                  0
seasons                 0
production_countries    0
age_certification       0
imdb_score              0
tmdb_score              0
dtype: int64

In [19]:
data_two=data

In [20]:
# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")


In [21]:
data.isna().sum()

release_year            0
runtime                 0
genres                  0
seasons                 0
production_countries    0
age_certification       0
imdb_score              0
tmdb_score              0
dtype: int64

In [22]:
#concaténer les champs
df["text"]=data["genres"]+""+data["production_countries"]+""+data["age_certification"]

In [23]:

model = SentenceTransformer('all-MiniLM-L6-v2')

In [24]:
embeddings = model.encode(df['text'].tolist(), show_progress_bar=True)

Batches: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:02<00:00,  2.76it/s]


In [27]:
X_num = data[["release_year", "runtime", "seasons", "imdb_score", "tmdb_score"]].values

In [32]:
# Fusion horizontale (concatène les deux matrices)
X = np.hstack([X_num, embeddings])

In [34]:
kmeans=KMeans(n_clusters=5,random_state=42)
labels=kmeans.fit_predict(X)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/Users/diallothiernosouleymane/Desktop/AppleTV/backend/venv/lib/python3.11/site-packages/threadpoolctl.py:1226: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [36]:
data["clusters"]=labels

/var/folders/hd/7x_lk_y95t96lcf50ts7cbph0000gn/T/ipykernel_34378/2026600077.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["clusters"]=labels


In [40]:
df["clusters"]=labels

In [41]:
df

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,text,clusters
0,tm1300,A Charlie Brown Christmas,MOVIE,When Charlie Brown complains about the overwhe...,1965,G,25,"['comedy', 'drama', 'family', 'music', 'animat...",['US'],0,tt0059026,8.3,40328.0,10.848,7.688,"['comedy', 'drama', 'family', 'music', 'animat...",2
1,tm71152,"It's the Great Pumpkin, Charlie Brown",MOVIE,"This classic ""Peanuts"" tale focuses on the thu...",1966,G,25,"['animation', 'comedy', 'family']",['US'],0,tt0060550,8.2,20299.0,12.458,7.474,"['animation', 'comedy', 'family']['US']G",2
2,tm2562,A Charlie Brown Thanksgiving,MOVIE,"Turkey, cranberries, pumpkin pie... and the Pe...",1973,G,25,"['family', 'animation', 'comedy']",['US'],0,tt0068359,7.7,11406.0,12.591,7.311,"['family', 'animation', 'comedy']['US']G",2
3,tm125274,"Be My Valentine, Charlie Brown",MOVIE,It's Valentine's Day again and Charlie Brown d...,1975,G,25,"['animation', 'family', 'romance', 'comedy']",['US'],0,tt0072687,7.4,2777.0,6.981,7.000,"['animation', 'family', 'romance', 'comedy']['...",2
4,tm106607,"It's the Easter Beagle, Charlie Brown",MOVIE,"Charlie Brown, Linus and the entire Peanuts ga...",1974,G,25,"['family', 'animation', 'comedy']",['US'],0,tt0071679,7.5,3001.0,6.172,6.600,"['family', 'animation', 'comedy']['US']G",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,ts328281,Dear Edward,SHOW,A 12-year-old boy survives a plane crash that ...,2023,TV-MA,0,['drama'],['US'],1.0,tt17543896,7.4,1324.0,37.810,2.000,['drama']['US']TV-MA,4
166,ts316895,The Reluctant Traveler with Eugene Levy,SHOW,Award-winning actor and nervous explorer Eugen...,2023,TV-PG,35,"['documentation', 'reality']",['GB'],1.0,tt15822846,7.8,473.0,13.892,6.800,"['documentation', 'reality']['GB']TV-PG",0
167,ts296808,Liaison,SHOW,This contemporary thriller explores how past m...,2023,TV-MA,51,"['crime', 'thriller']","['FR', 'GB']",1.0,tt14792896,6.3,549.0,84.911,6.100,"['crime', 'thriller']['FR', 'GB']TV-MA",3
168,ts374301,Super League: The War For Football,SHOW,"The past, present, and future of European foot...",2023,TV-MA,0,"['documentation', 'sport']",['US'],1.0,tt24587530,7.4,414.0,6.800,6.700,"['documentation', 'sport']['US']TV-MA",4


In [42]:
df.to_csv('data_final.csv', index=False)